<a href="https://colab.research.google.com/github/piercepatrick/Titans_2019_FF_Analysis/blob/master/Tannehilll_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. How did Ryan Tannehill compare to other top Quarterbacks?

2. How much did the Marcus Mariota -> Ryan Tannehill switch in Week 7 affect the offense's Fantasy Footabll performance?

3. Did Ryan Tannehill over-perform in the 2019-2020 season and is he due for a regression in the 2020-2021 season?


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
WEEKLY_BASE_URL = '/content/drive/My Drive/DS/Titans FF Analysis/data_v2/weekly/2019/week{}.csv'
YEARLY_BASE_URL = '/content/drive/My Drive/DS/Titans FF Analysis/data_v2/yearly/2019.csv'

In [ ]:
def generate_df():
  df = pd.DataFrame()
  for week in range(1, 18):
    weekly_df = pd.read_csv(WEEKLY_BASE_URL.format(week))
    weekly_df['Week'] = week
    df = pd.concat([df, weekly_df])
  return df

df = generate_df()
df

,Player,Pos,Tm,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgt,ReceivingYds,ReceivingTD,FL,PPRFantasyPoints,StandardFantasyPoints,HalfPPRFantasyPoints,Week
0,Lamar Jackson,QB,BAL,324.0,5.0,0.0,20.0,17.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,33.56,33.56,33.56,1
1,Dak Prescott,QB,DAL,405.0,4.0,0.0,32.0,25.0,4.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,33.40,33.40,33.40,1
2,Robert Griffin,QB,BAL,55.0,1.0,0.0,6.0,6.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,7.10,7.10,7.10,1
3,Patrick Mahomes,QB,KAN,378.0,3.0,0.0,33.0,25.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,27.32,27.32,27.32,1
4,Kirk Cousins,QB,MIN,98.0,1.0,0.0,10.0,8.0,6.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,14.32,14.32,14.32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,Tavon Austin,WR,DAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.10,0.10,0.60,17
301,Blake Bell,TE,KAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,0.0,1.10,0.10,0.60,17
302,Jamal Agnew,CB,DET,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,-2.0,0.0,0.0,0.80,-0.20,0.30,17
303,Trevor Davis,WR,MIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,17


In [ ]:
df = df.drop(['PPRFantasyPoints', 'StandardFantasyPoints'], axis = 1)

In [ ]:
df.rename(columns = {'HalfPPRFantasyPoints': 'FantasyPoints'}, inplace = True)

In [ ]:
qb_df = df.loc[df['Pos'] == 'QB']
qb_df

,Player,Pos,Tm,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgt,ReceivingYds,ReceivingTD,FL,FantasyPoints,Week
0,Lamar Jackson,QB,BAL,324.0,5.0,0.0,20.0,17.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,33.56,1
1,Dak Prescott,QB,DAL,405.0,4.0,0.0,32.0,25.0,4.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,33.40,1
2,Robert Griffin,QB,BAL,55.0,1.0,0.0,6.0,6.0,4.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,7.10,1
3,Patrick Mahomes,QB,KAN,378.0,3.0,0.0,33.0,25.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,27.32,1
4,Kirk Cousins,QB,MIN,98.0,1.0,0.0,10.0,8.0,6.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,14.32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,David Blough,QB,DET,122.0,0.0,1.0,29.0,12.0,2.0,11.0,0.0,1.0,1.0,19.0,1.0,0.0,12.38,17
38,Sean Mannion,QB,MIN,126.0,0.0,2.0,21.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.96,17
39,Will Grier,QB,CAR,4.0,0.0,1.0,8.0,1.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,-3.34,17
124,Taysom Hill,QB,NOR,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,1.0,1.0,45.0,1.0,0.0,11.40,17


In [ ]:
qb_df.drop(['Rec', 'Tgt', 'ReceivingYds', 'ReceivingTD'], axis = 1, inplace = True)
qb_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Player,Pos,Tm,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,FL,FantasyPoints,Week
0,Lamar Jackson,QB,BAL,324.0,5.0,0.0,20.0,17.0,3.0,6.0,0.0,0.0,33.56,1
1,Dak Prescott,QB,DAL,405.0,4.0,0.0,32.0,25.0,4.0,12.0,0.0,0.0,33.40,1
2,Robert Griffin,QB,BAL,55.0,1.0,0.0,6.0,6.0,4.0,9.0,0.0,0.0,7.10,1
3,Patrick Mahomes,QB,KAN,378.0,3.0,0.0,33.0,25.0,1.0,2.0,0.0,0.0,27.32,1
4,Kirk Cousins,QB,MIN,98.0,1.0,0.0,10.0,8.0,6.0,4.0,1.0,0.0,14.32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,David Blough,QB,DET,122.0,0.0,1.0,29.0,12.0,2.0,11.0,0.0,0.0,12.38,17
38,Sean Mannion,QB,MIN,126.0,0.0,2.0,21.0,12.0,1.0,0.0,0.0,1.0,-0.96,17
39,Will Grier,QB,CAR,4.0,0.0,1.0,8.0,1.0,3.0,5.0,0.0,1.0,-3.34,17
124,Taysom Hill,QB,NOR,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,11.40,17


In [ ]:
qb_df['Y/A'] = qb_df['PassingYds'] / qb_df['PassingAtt']
qb_df['Y/A'] = qb_df['Y/A'].apply(lambda x: round(x, 2))
qb_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Player,Pos,Tm,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,FL,FantasyPoints,Week,Y/A
0,Lamar Jackson,QB,BAL,324.0,5.0,0.0,20.0,17.0,3.0,6.0,0.0,0.0,33.56,1,16.20
1,Dak Prescott,QB,DAL,405.0,4.0,0.0,32.0,25.0,4.0,12.0,0.0,0.0,33.40,1,12.66
2,Robert Griffin,QB,BAL,55.0,1.0,0.0,6.0,6.0,4.0,9.0,0.0,0.0,7.10,1,9.17
3,Patrick Mahomes,QB,KAN,378.0,3.0,0.0,33.0,25.0,1.0,2.0,0.0,0.0,27.32,1,11.45
4,Kirk Cousins,QB,MIN,98.0,1.0,0.0,10.0,8.0,6.0,4.0,1.0,0.0,14.32,1,9.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,David Blough,QB,DET,122.0,0.0,1.0,29.0,12.0,2.0,11.0,0.0,0.0,12.38,17,4.21
38,Sean Mannion,QB,MIN,126.0,0.0,2.0,21.0,12.0,1.0,0.0,0.0,1.0,-0.96,17,6.00
39,Will Grier,QB,CAR,4.0,0.0,1.0,8.0,1.0,3.0,5.0,0.0,1.0,-3.34,17,0.50
124,Taysom Hill,QB,NOR,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,11.40,17,NaN


In [ ]:
qb_df.groupby('Player')['FantasyPoints'].sum().sort_values(ascending=False).head(30)

Player
Lamar Jackson        415.68
Dak Prescott         335.78
Russell Wilson       326.60
Deshaun Watson       318.48
Jameis Winston       297.36
Patrick Mahomes      285.04
Josh Allen           282.56
Kyler Murray         279.28
Aaron Rodgers        272.38
Carson Wentz         271.86
Matt Ryan            265.34
Tom Brady            261.68
Jared Goff           247.52
Derek Carr           244.36
Jimmy Garoppolo      243.32
Kirk Cousins         242.42
Ryan Fitzpatrick     239.46
Philip Rivers        233.50
Baker Mayfield       227.18
Drew Brees           224.76
Gardner Minshew      223.24
Ryan Tannehill       222.18
Jacoby Brissett      215.18
Daniel Jones         210.98
Mitchell Trubisky    200.82
Andy Dalton          199.16
Sam Darnold          183.16
Kyle Allen           177.48
Matthew Stafford     166.56
Mason Rudolph        108.80
Name: FantasyPoints, dtype: float64